# 연구 3단계 (GPU 버전): MNIST 데이터셋 환경에서 노이즈와 GMM 복잡도 관계 분석

## ⚠️ 실행 전 필수 확인 사항 ⚠️
본 노트북은 GPU 가속을 위해 NVIDIA RAPIDS `cuML` 라이브러리를 사용합니다. 실행을 위해서는 NVIDIA GPU 및 관련 `conda` 환경 설정이 필요합니다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from cuml.mixture import GaussianMixture # cuML GMM
from cuml.decomposition import PCA       # cuML PCA
from tensorflow.keras.datasets import mnist
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

## 1. 실험 조건 설정

In [ ]:
N_RUNS = 30
NOISE_LEVELS = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
N_COMPONENTS_RANGE = range(1, 26)
N_PCA_COMPONENTS = 32

## 2. MNIST 데이터 로딩 및 BIC 계산 함수 정의

In [ ]:
# MNIST 데이터 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_all = np.concatenate([x_train, x_test], axis=0)
x_all_flat = x_all.reshape(-1, 28*28) / 255.0

def calculate_bic(gmm, data):
    """cuML GMM 모델에 대한 BIC 점수를 수동으로 계산합니다."""
    n_samples, n_features = data.shape
    n_components = gmm.n_components
    
    cov_params = n_components * n_features * (n_features + 1) / 2.
    mean_params = n_components * n_features
    weight_params = n_components - 1
    n_params = cov_params + mean_params + weight_params
    
    log_likelihood = gmm.score(data)
    bic = n_params * np.log(n_samples) - 2 * log_likelihood * n_samples
    return bic

## 3. 핵심 연구 로직 실행 (GPU 가속)

In [ ]:
results = []
pca = PCA(n_components=N_PCA_COMPONENTS) # cuML PCA 사용

for noise in tqdm(NOISE_LEVELS, desc="Noise Levels"):
    optimal_k_for_runs = []
    for run in tqdm(range(N_RUNS), desc=f"Runs for Noise {noise}", leave=False):
        # 1. 원본 이미지에 노이즈 주입
        noise_matrix = np.random.randn(*x_all_flat.shape) * noise
        x_all_flat_noisy = np.clip(x_all_flat + noise_matrix, 0, 1)
        x_all_flat_noisy = x_all_flat_noisy.astype(np.float32)
        
        # 2. PCA를 이용해 차원 축소 (GPU)
        latent_vectors = pca.fit_transform(x_all_flat_noisy)
        
        # 3. 각 k에 대해 GMM 학습 및 BIC 계산 (GPU)
        bics = []
        for k in N_COMPONENTS_RANGE:
            gmm = GaussianMixture(n_components=k, reg_covar=1e-6)
            gmm.fit(latent_vectors)
            bic_score = calculate_bic(gmm, latent_vectors)
            bics.append(bic_score)
            
        best_k = N_COMPONENTS_RANGE[np.argmin(bics)]
        optimal_k_for_runs.append(best_k)
        
    results.append({
        'noise_level': noise,
        'mean_k_opt': np.mean(optimal_k_for_runs),
        'std_k_opt': np.std(optimal_k_for_runs)
    })

results_df_mnist = pd.DataFrame(results)

## 4. 결과 분석 및 시각화

In [ ]:
print("--- MNIST 실험 결과 (GPU) ---")
print(results_df_mnist[['noise_level', 'mean_k_opt', 'std_k_opt']])

plt.figure(figsize=(12, 7))
plt.errorbar(
    results_df_mnist['noise_level'], 
    results_df_mnist['mean_k_opt'], 
    yerr=results_df_mnist['std_k_opt'],
    fmt='-D', capsize=5, label='평균 최적 k (오차막대: 1-std)'
)
plt.title('노이즈 수준에 따른 GMM 최적 컴포넌트 개수 변화 (MNIST 잠재 공간, GPU)', fontsize=16)
plt.xlabel('노이즈 수준 (Noise Level)', fontsize=12)
plt.ylabel(f'최적 컴포넌트 개수 (평균, N={N_RUNS}회 실행)', fontsize=12)
plt.xticks(NOISE_LEVELS)
plt.grid(True, which='both', linestyle='--')
plt.legend()
plt.show()